In [1]:
library(tidyverse)
library(RPostgreSQL)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "SQL")
list.files(sql_dir)

[1] "cohort.sql"     "data.sql"       "population.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic", user = "lcp_db_user")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
sql <- readr::read_file(file.path(sql_dir, "cohort.sql")) %>%
    make_view_sql("cohort", "materialized view")

In [6]:
dbSendQuery(con, sql)

<PostgreSQLResult>

In [7]:
sql <- "
select * from cohort limit 10;
"
dbGetQuery(con, sql)

icustay_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time
200003,SICU,2199-08-02 19:50:04,2199-08-08 17:09:18,1,48.29627,1,2199-08-03
200014,SICU,2105-02-16 23:16:48,2105-02-18 16:53:29,1,84.73004,1,2105-02-18
200030,MICU,2150-11-13 14:08:02,2150-11-19 17:51:59,1,54.19308,1,2150-11-16
200033,SICU,2198-08-07 17:56:17,2198-08-21 14:59:18,1,67.14657,1,NA
200036,MICU,2181-08-26 16:38:25,2181-08-29 16:28:48,1,74.93578,1,NA
200075,MICU,2159-09-23 00:13:20,2159-09-25 01:55:17,1,83.64320,1,2159-09-25
200087,MICU,2196-08-30 11:19:49,2196-09-03 17:40:09,1,64.32851,1,2196-08-31
200131,MICU,2176-10-30 12:05:18,2176-11-07 17:34:37,1,69.31159,1,NA
200143,MICU,2191-04-01 21:45:49,2191-04-28 16:32:23,1,62.24888,1,NA
200199,MICU,2186-05-27 23:36:28,2186-05-28 15:19:11,1,300.00379,1,NA


In [8]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE